<a href="https://colab.research.google.com/github/SathyaSudha-96/Deep-Learning-2021-22/blob/main/IDL_Assignment_4_SathyaSudhaMurugan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras import layers
import time
from keras.utils.vis_utils import plot_model
import tensorboard
from datetime import datetime

In [ ]:
cifar_data = tf.keras.datasets.cifar10
(train_images, train_labels),(test_images,test_labels) = cifar_data.load_data()

In [ ]:
print('Shape of Training Images',train_images.shape)
print('Shape of Training Labels',train_labels.shape)
print('Shape of Testing Images',test_images.shape)
print('Shape of Testing Labels',test_labels.shape)

In [ ]:
figure, ax = plt.subplots(2,4)
figure.suptitle("CIFAR_10 Data Samples")
ax[0,0].imshow(train_images[1])
ax[0,1].imshow(train_images[800])
ax[0,2].imshow(train_images[30])
ax[0,3].imshow(train_images[40])
ax[1,0].imshow(train_images[5])
ax[1,1].imshow(train_images[100])
ax[1,2].imshow(train_images[7])
ax[1,3].imshow(train_images[20])

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(
    (train_images.astype(np.float32) / 255, train_labels.reshape(-1).astype(np.int32)))
train_data = train_data.shuffle(buffer_size=50000).batch(128)

test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.astype(np.float32) / 255, test_labels.reshape(-1).astype(np.int32))).batch(128)

Each convolution is preceded by batch normalization and relu activation.

In [ ]:
def bn_relu_conv(tensor,filters, kernel_size):
    x = layers.BatchNormalization()(tensor)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size, strides =1, padding="same")(x)
    return x

In [ ]:
  def dense_block(tensor,filter, r):
    for _ in range(r):
      x = bn_relu_conv(tensor, filters=4*filter, kernel_size=1) #for 1*1 convolution, the number for filters if 4*filters
      x = bn_relu_conv(tensor=x, filters=filter, kernel_size=3)
      tensor = layers.Concatenate()([tensor, x])
    return tensor

Transition Layer 

In [ ]:
 def transition_block(x):
    f = int(tf.keras.backend.int_shape(x)[-1] // 2)
    x = bn_relu_conv(tensor=x, filters=f, kernel_size=1)
    x = layers.AvgPool2D(2, strides=2, padding='same')(x)
    return x

In [ ]:
def densenet_121():
  repetitions = 6, 12, 24, 16
  filter = 32
  inputs = tf.keras.layers.Input((32, 32, 3))
  x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
  x = layers.MaxPool2D(3, strides=2, padding='same')(x)
  for r in repetitions:
    d = dense_block(x, filter, r)
    x = transition_block(d)
  x = layers.GlobalAvgPool2D()(d)
  output = layers.Dense(1000, activation='softmax')(x)
  model = keras.models.Model(inputs, output, name = "DenseNet_121")
  return model
densenet_121_model = densenet_121()  

In [ ]:
def densenet_169():
  repetitions = 6, 12, 32, 32
  filter = 32
  inputs = tf.keras.layers.Input((32, 32, 3))
  x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
  x = layers.MaxPool2D(3, strides=2, padding='same')(x)
  for r in repetitions:
    d = dense_block(x, filter, r)
    x = transition_block(d)
  x = layers.GlobalAvgPool2D()(d)
  output = layers.Dense(1000, activation='softmax')(x)
  model = keras.models.Model(inputs, output, name = "DenseNet_121")
  return model
densenet_169_model = densenet_169()  

In [ ]:
def densenet_201():
  repetitions = 6, 12, 48, 32
  filter = 32
  inputs = tf.keras.layers.Input((32, 32, 3))
  x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
  x = layers.MaxPool2D(3, strides=2, padding='same')(x)
  for r in repetitions:
    d = dense_block(x, filter, r)
    x = transition_block(d)
  x = layers.GlobalAvgPool2D()(d)
  output = layers.Dense(1000, activation='softmax')(x)
  model = keras.models.Model(inputs, output, name = "DenseNet_121")
  return model
densenet_201_model = densenet_201()  

In [ ]:
@tf.function
def train_step(images, labels, model,optimizer):
  #logits is false as we have used softmax in output
  loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
  with tf.GradientTape() as tape:
    logits = model(images)
    loss = loss_fn(labels, logits)
  #Calculate gradients using gradient tape
  gradients = tape.gradient(loss, model.trainable_variables)
  #Assign new variables to the model using optimizer instead of sub assign
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss, logits

In [ ]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

In [ ]:
def training_loop(epochs, model,optimizer):
  train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
  start_time = time.time()
  for epoch in range(epochs):
    print("Starting epoch", epoch+1)
    for step, (image_batch, label_batch) in enumerate(train_data):
      loss, logits = train_step(image_batch, label_batch,model,optimizer)
      #Calculate  traning accuracy
      train_acc_metric(label_batch, logits)
    
      if not step % 100:  
        print("Loss: {} Training Accuracy: {}".format(loss, train_acc_metric.result()))
        train_acc_metric.reset_states()
    print("Time taken: %.2fs" % (time.time() - start_time))
      

In [ ]:
training_loop(10, densenet_121_model, optimizer = tf.optimizers.Adam())

Without @tf.function, it took 1824.09 seconds (31.4 minutes)

With @tf.function, it took only 706.28 seconds (11.77 minutes) which is less than half of without @tf.function.⚡

In [ ]:
test_acc_metric = tf.metrics.SparseCategoricalAccuracy()
for img_batch, lbl_batch in test_data:
    test_acc_metric(lbl_batch, densenet_121_model(img_batch))

test_acc_metric.result()

In [ ]:
training_loop(50, densenet_169_model, optimizer = tf.optimizers.Adam())

In [ ]:
test_acc_metric = tf.metrics.SparseCategoricalAccuracy()
for img_batch, lbl_batch in test_data:
    test_acc_metric(lbl_batch, densenet_169_model(img_batch))

test_acc_metric.result()

TensorBoard to view the computational graph

In [ ]:
def training_loop_TensorBoard(epochs, model,optimizer):
  train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
  start_time = time.time()
  for epoch in range(epochs):
    print("Starting epoch", epoch+1)
    for step, (image_batch, label_batch) in enumerate(train_data):
      #Enabling the tracing
      tf.summary.trace_on(graph=True, profiler=True)
      loss, logits = train_step(image_batch, label_batch,model,optimizer)
      #Calculate  traning accuracy
      train_acc_metric(label_batch, logits)
      with writer.as_default():
        #Exporting the trace 
        tf.summary.trace_export(
        name="my_func_trace",
        step=0,
        profiler_outdir=logdir)
    
      if not step % 100:  
        print("Loss: {} Training Accuracy: {}".format(loss, train_acc_metric.result()))
        train_acc_metric.reset_states()
    print("Time taken: %.2fs" % (time.time() - start_time))
training_loop_TensorBoard(1, densenet_201_model, optimizer = tf.optimizers.Adam())

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/func

Training pipeline with high level keras

In [ ]:
def densenet_264():
  repetitions = 6, 12, 64, 48
  filter = 32
  inputs = tf.keras.layers.Input((32, 32, 3))
  x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
  x = layers.MaxPool2D(3, strides=2, padding='same')(x)
  for r in repetitions:
    d = dense_block(x, filter, r)
    x = transition_block(d)
  x = layers.GlobalAvgPool2D()(d)
  output = layers.Dense(1000, activation='softmax')(x)
  model = keras.models.Model(inputs, output, name = "DenseNet_121")
  return model
densenet_264_model = densenet_264()  
#densenet_264_model.summary()

optimizer = tf.optimizers.Adam()

loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

accuracy_metric = tf.metrics.SparseCategoricalAccuracy()

densenet_264_model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])

densenet_264_model.fit(train_data, steps_per_epoch=1000, epochs=2, validation_data=test_data)
